In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

We have transformed the dataset section from the PDF file authored by Johnson et al. into an Excel file. 
Following this, we conducted the subsequent processing steps.

# 1. Excel Dataset Processing

- Nodes

In [3]:
node_skiprows = list(range(1, 5))

In [4]:
# Import Node
node_df = pd.read_excel('Data/Vaccination_data.xlsx').iloc[:1326]; 
node_df.head()

,selector,Unnamed: 1,polarity,x,Unnamed: 4,Unnamed: 5,y,Unnamed: 7,Unnamed: 8,banner_cdc,Unnamed: 10,Unnamed: 11,fan_count,degree,in_degree
0,a_001,NaN,anti_vaccines,1567.709717,NaN,NaN,-24218.750000,NaN,NaN,False,NaN,NaN,94.0,0.0,0.0
1,a_00100,NaN,anti_vaccines,-1835.667725,NaN,NaN,25781.250000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,a_001001,NaN,anti_vaccines,160.108688,NaN,NaN,-749.809448,NaN,NaN,False,NaN,NaN,22175.0,3.0,3.0
3,a_001002,NaN,anti_vaccines,-137.362717,NaN,NaN,-676.494263,NaN,NaN,True,NaN,NaN,423.0,7.0,6.0
4,a_001003,NaN,anti_vaccines,181.973236,NaN,NaN,-107.275665,NaN,NaN,True,NaN,NaN,289.0,8.0,8.0


In [5]:
# Drop Extra Columns
node_col_drop = ['Unnamed: 1', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 10', 'Unnamed: 11']
for col_drop in node_col_drop:
    node_df = node_df.drop(col_drop, axis = 1)
node_df.head()

,selector,polarity,x,y,banner_cdc,fan_count,degree,in_degree
0,a_001,anti_vaccines,1567.709717,-24218.750000,False,94.0,0.0,0.0
1,a_00100,anti_vaccines,-1835.667725,25781.250000,NaN,0.0,0.0,0.0
2,a_001001,anti_vaccines,160.108688,-749.809448,False,22175.0,3.0,3.0
3,a_001002,anti_vaccines,-137.362717,-676.494263,True,423.0,7.0,6.0
4,a_001003,anti_vaccines,181.973236,-107.275665,True,289.0,8.0,8.0


- Edges

In [6]:
# Import Edges
edge_df = pd.read_excel('Data/Vaccination_data.xlsx', header=[2654]).iloc[:7675]; 
edge_df

,source,Unnamed: 1,target,polarity,Unnamed: 4,T1,Unnamed: 6,Unnamed: 7,capture_date,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,p_00667,NaN,n_00381,pro_vaccines,NaN,FALSE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,a_00502,NaN,a_001280,an#_vaccines,NaN,TRUE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,p_00914,NaN,p_001274,pro_vaccines,NaN,TRUE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,p_00914,NaN,p_001233,pro_vaccines,NaN,TRUE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,p_00914,NaN,p_001142,pro_vaccines,NaN,TRUE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670,n_00619,NaN,a_001217,neutral,NaN,TRUE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
7671,n_00619,NaN,a_001264,neutral,NaN,TRUE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
7672,n_00619,NaN,a_001130,neutral,NaN,TRUE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
7673,n_00619,NaN,a_001281,neutral,NaN,TRUE,NaN,NaN,2019-10-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Drop Extra Columns
edge_col_drop = ['Unnamed: 1', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14']
for col_drop in edge_col_drop:
    edge_df = edge_df.drop(col_drop, axis = 1)
# Drop Extra Rows
edge_df = edge_df[edge_df.source != 'source']
edge_df.head()

,source,target,polarity,T1,capture_date
0,p_00667,n_00381,pro_vaccines,FALSE,2019-10-05 00:00:00
1,a_00502,a_001280,an#_vaccines,TRUE,2019-10-05 00:00:00
2,p_00914,p_001274,pro_vaccines,TRUE,2019-10-05 00:00:00
3,p_00914,p_001233,pro_vaccines,TRUE,2019-10-05 00:00:00
4,p_00914,p_001142,pro_vaccines,TRUE,2019-10-05 00:00:00


In [8]:
# T1: edge_df1    T2: edge_df2
edge_df2 = edge_df
edge_df1 = edge_df[edge_df.T1 == 'TRUE']

- fan size

In [9]:
# Import Size
size_df = pd.read_excel('Data/Vaccination_data.xlsx', header=[10330]).iloc[:1359]; 
size_df;

In [10]:
# Drop Extra Columns
size_col_drop = ['Unnamed: 1', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14']
for col_drop in size_col_drop:
    size_df = size_df.drop(col_drop, axis = 1)
# Drop Extra Rows
size_df = size_df[size_df.selector != 'selector']
size_df.head()

,selector,key,value_t1,value_t2
0,a_00203,fan_count,260,261
1,a_00816,fan_count,3423,3507
2,p_00667,fan_count,1075,1400
3,p_00141,fan_count,720,734
4,a_00267,fan_count,116,117


In [11]:
fan_size_t1 = []; fan_size_t2 = []
for selector in node_df.selector.values:
    fan_size_t1.append(size_df[size_df.selector == selector].value_t1.values[0])
    fan_size_t2.append(size_df[size_df.selector == selector].value_t2.values[0])

In [12]:
node_df.insert(8, "fan_size_t1", fan_size_t1)
node_df.insert(9, "fan_size_t2", fan_size_t2)

In [13]:
node_df.head()

,selector,polarity,x,y,banner_cdc,fan_count,degree,in_degree,fan_size_t1,fan_size_t2
0,a_001,anti_vaccines,1567.709717,-24218.750000,False,94.0,0.0,0.0,78,94
1,a_00100,anti_vaccines,-1835.667725,25781.250000,NaN,0.0,0.0,0.0,5082,0
2,a_001001,anti_vaccines,160.108688,-749.809448,False,22175.0,3.0,3.0,20773,22175
3,a_001002,anti_vaccines,-137.362717,-676.494263,True,423.0,7.0,6.0,409,423
4,a_001003,anti_vaccines,181.973236,-107.275665,True,289.0,8.0,8.0,294,289


# 2. Create Graphs

In [14]:
G1 = nx.DiGraph(); G2 = nx.DiGraph()

In [15]:
# Add nodes
node_name = node_df[["selector"]].values.tolist(); node_name = sum(node_name, [])
node_polarity = node_df[["polarity"]].values.tolist(); node_polarity = sum(node_polarity, [])

node_pos = node_df[["x", "y"]].values.tolist(); node_pos = [tuple(x) for x in node_pos]

fan_size_t1 = node_df[["fan_size_t1"]].values.tolist(); fan_size_t1 = sum(fan_size_t1, [])
fan_size_t2 = node_df[["fan_size_t2"]].values.tolist(); fan_size_t2 = sum(fan_size_t2, [])

for i in range(len(node_name)):
    G1.add_node(node_name[i], polarity = node_polarity[i], pos = node_pos[i], fan_size = fan_size_t1[i])
    G2.add_node(node_name[i], polarity = node_polarity[i], pos = node_pos[i], fan_size = fan_size_t2[i])

In [16]:
# Add edges
edge1 = edge_df1[["source", "target"]].values.tolist(); 
edge2 = edge_df2[["source", "target"]].values.tolist();

for i in range(len(edge1)):
    G1.add_edge(edge1[i][0], edge1[i][1], weight = node_df[node_df["selector"] == edge1[i][0]][['fan_size_t1']].values[0][0] * node_df[node_df["selector"] == edge1[i][1]][['fan_size_t1']].values[0][0])

for i in range(len(edge2)):
    G2.add_edge(edge2[i][0], edge2[i][1], weight = node_df[node_df["selector"] == edge2[i][0]][['fan_size_t2']].values[0][0] * node_df[node_df["selector"] == edge2[i][1]][['fan_size_t2']].values[0][0])

# 3. Save Graphs

## 3.1. gpickle

In [17]:
nx.write_gpickle(G1, "Data/Graphs/G1.gpickle")
nx.write_gpickle(G2, "Data/Graphs/G2.gpickle")

In [18]:
nx.info(G1), nx.info(G2)

('DiGraph with 1326 nodes and 5163 edges',
 'DiGraph with 1326 nodes and 7484 edges')

In [19]:
G1.nodes['a_001'], G2.nodes['a_001']

({'polarity': 'anti_vaccines',
  'pos': (1567.709717, -24218.75),
  'fan_size': 78},
 {'polarity': 'anti_vaccines',
  'pos': (1567.709717, -24218.75),
  'fan_size': 94})

In [20]:
G1.edges[('a_001002', 'a_001251')], G2.edges[('a_001002', 'a_001251')]

({'weight': 1510437}, {'weight': 1777446})

## 3.2. CSV

In [21]:
import csv

In [22]:
def write_nodes_csv(G, nodes, attributes, address):
    
    file = open(address, 'w', newline ='')
    header = ['node'] + attributes
    
    with file:
        # identifying header 
        writer = csv.DictWriter(file, fieldnames = header)
        
        # writing data row-wise into the csv file
        writer.writeheader()
        
        for node in nodes:
            node_attributes = G.nodes[node]
            node_attributes['node'] = node
            writer.writerow(node_attributes)

In [23]:
write_nodes_csv(G1, list(G1.nodes), ['polarity', 'fan_size', 'pos'], "Data/Graphs/G1_nodes.csv")

In [24]:
write_nodes_csv(G2, list(G2.nodes), ['polarity', 'fan_size', 'pos'], "Data/Graphs/G2_nodes.csv")

In [25]:
def write_edges_csv(G, edges, attributes, address):
    
    file = open(address, 'w', newline ='')
    header = ['source_node', 'target_node'] + attributes
    
    with file:
        # identifying header 
        writer = csv.DictWriter(file, fieldnames = header)
        
        # writing data row-wise into the csv file
        writer.writeheader()
        
        for edge in edges:
            edge_attributes = G.edges[edge]
            edge_attributes['source_node'] = edge[0]
            edge_attributes['target_node'] = edge[1]
            writer.writerow(edge_attributes)

In [26]:
write_edges_csv(G1, list(G1.edges), ['weight'], "Data/Graphs/G1_edges.csv")

In [27]:
write_edges_csv(G2, list(G2.edges), ['weight'], "Data/Graphs/G2_edges.csv")